## DeepSpeed 加速框架

DeepSpeed是微软开源的一款针对大规模模型分布式训练的工具，官方地址:

开源库：https://github.com/microsoft/DeepSpeed

官方示例：https://github.com/microsoft/DeepSpeedExamples

官方文档：https://www.deepspeed.ai/getting-started/

接下来我们依据教程和示例，将DeepSpeed应用在cifar10模型训练的代码上

首先需要更改argparse设置部分。根据DeepSpeed官方文档的指引，我们需要添加DeepSpeed训练的config设置文件，并将其置入argparse中，如下：

In [ ]:
import argparse
import utils
import deepspeed

def get_args_parser():
    parser = argparse.ArgumentParser('Distributed training')
    # parser.add_argument('--local_rank', type=int, default=0)
    parser.add_argument('--world_size', type=int, default=1)
    parser.add_argument('--backend', type=str, default='nccl')
    parser.add_argument("--dist-url", type=str, default="env://")
    parser.add_argument("--sync-bn", action="store_true")
    parser.add_argument("--use-deepspeed", action="store_true")
    
    args, _ = parser.parse_known_args()
    #! ---------------------------------------------------------------
    if args.use_deepspeed:
        parser = deepspeed.add_config_arguments(parser)
    #! ---------------------------------------------------------------
    
    return parser.parse_args()


def main():
    args = get_args_parser()
    #! ---------------------------------------------------------------
    if args.use_deepspeed:
        args.deepspeed_config = "./config/deepspeed_cifar_config.json"
    #! ---------------------------------------------------------------
    utils.init_distributed_mode(args)

其中，config文件我们使用了[最简单的版本](./config/deepspeed_cifar_config.json)，如下。在实际代码运行中，应根据代码执行的具体情况调整config文件的设置。

```json
{
  "train_batch_size": 64,
  "gradient_accumulation_steps": 1,
  "optimizer": {
    "type": "Adam",
    "params": {
      "lr": 0.001
    }
  },
  "fp16": {
    "enabled": false
  },
  "zero_optimization": {
    "stage": 2
  }
}
```

其次，我们需要更新分布式环境的初始化方式。根据官方文档（https://www.deepspeed.ai/getting-started/#writing-deepspeed-models），使用deepspeed初始化需要代替掉之前我们所使用的`torch.distributed.init_process_group(...)`方法，改用`deepspeed.init_distributed()`方法，即：

In [ ]:
import torch

def init_distributed_mode(args):
    '''...'''
    torch.cuda.set_device(args.gpu)
    args.dist_backend = "nccl"
    print(f"| distributed init (rank {args.rank}): {args.dist_url}", flush=True)

    if args.use_deepspeed:
        import deepspeed
        deepspeed.init_distributed(dist_backend=args.dist_backend, init_method=args.dist_url)
    else:
        torch.distributed.init_process_group(backend=args.dist_backend, init_method=args.dist_url, world_size=args.world_size, rank=args.rank)

接下来，我们使用DeepSpeed框架对模型进行封装：

In [ ]:
'''
    criteria = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    
    # modify here
    if args.use_deepspeed:
        net, optimizer, _, _ = deepspeed.initialize(args=args, model=net, optimizer=optimizer)

    for epoch in range(10):
        # start training
'''

这段代码的核心是`deepspeed.initialize()`这个函数。

接下来，模型的前向过程也需要做一定的调整。具体来说，利用`deepspeed.initialize()`函数封装过的模型，会自动地执行优化器清零（即`optimizer.zero_grad()`）和学习率规划的步骤（ learning rate scheduler），因此前向过程会略简单一些：

In [ ]:
'''
for i, (inputs, labels) in enumerate(train_loader):
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = net(inputs)
    loss = criteria(outputs, labels)
    
    loss_sum += loss.item()
    predict = torch.argmax(outputs, dim=1)
    acc_sum += torch.sum(predict == labels).item()
    
    if args.use_deepspeed:
        net.backward(loss)
        net.step()
    else:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
'''

现在一个非常简单的带有deepspeed训练框架的cifar代码就搭建完成了。接下来我们在多卡上试一试启动deepspeed训练的效果：

In [4]:
!torchrun --nproc_per_node=8 main.py --use-deepspeed

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 0): env://
[2023-04-20 07:35:29,185] [INFO] [comm.py:586:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
| distributed init (rank 6): env://
| distributed init (rank 2): env://
| distributed init (rank 1): env://
| distributed init (rank 7): env://
| distributed init (rank 3): env://
| distributed init (rank 5): env://
| distributed init (rank 4): env://
Cifar dataset already exist in './data', skip download
[2023-04-20 07:35:44,801] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.8.3+4d27225f, git-hash=4d27225f, git-branch=master
[2023-04-20 07:35:45,020] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enable

##### 下一节中，我们重点关注DeepSpeed下所给出的一些API方法